In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
data_path = '../data/ignore/'
old_gpu = pd.read_csv(data_path + 'gpu_cleaned.csv', index_col='name')
new_gpu = pd.read_csv(data_path + 'best_gpu_cleaned.csv', index_col='Name')

In [4]:
# Make the name index for faster lookup
trim = {'amd':'', 'intel':'', 'apple':'', 'qualcomm':'', 'nvidia':''}
old_gpu.index = old_gpu.index.astype(str).str.lower().str.strip()
new_gpu.index = new_gpu.index.astype(str).str.lower().str.strip()
for key in trim:
    old_gpu.index = old_gpu.index.str.replace(key, trim[key]).str.strip()
    new_gpu.index = new_gpu.index.str.replace(key, trim[key]).str.strip()

In [5]:
gpu_key = pd.read_csv(data_path + 'gpu_key.csv')
gpu_key.drop(columns=['Column'], inplace=True)

In [6]:
gpu_key['b_2'] = gpu_key['b_2'].str.lower().str.strip()
gpu_key['b_1'] = gpu_key['b_1'].str.lower().str.strip()

In [7]:
# merge the name according to the cpu_key file

# the old cpu file first
for i in range(len(old_gpu)):
    cur_name = old_gpu.index[i]
    corresponding_name = gpu_key[gpu_key['b_1'] == cur_name]['0']
    if len(corresponding_name) > 0:
        old_gpu.index.values[i] = corresponding_name.values[0]
    else:
        old_gpu.index.values[i] = np.nan

# the new cpu file
for i in range(len(new_gpu)):
    cur_name = new_gpu.index[i]
    corresponding_name = gpu_key[gpu_key['b_2'] == cur_name]['0']
    if len(corresponding_name) > 0:
        new_gpu.index.values[i] = corresponding_name.values[0]
    else:
        new_gpu.index.values[i] = np.nan

In [8]:
pd_out = pd.DataFrame()

Merge the columns that are important according to the key file

In [9]:
# Name: just copy the name from key
pd_out['Name'] = gpu_key['0']

In [10]:
# manufactoring process / process size

pd_out['Process Size (nm)'] = np.nan
num = 0

for i in range(len(gpu_key)):
    cur_name = gpu_key['0'][i]
    left = old_gpu[old_gpu.index == cur_name]['Manufacturing process'].astype(str)
    right = new_gpu[new_gpu.index == cur_name]['Process Size'].astype(str)
    if len(left) > 0 and len(right) == 0:
        pd_out['Process Size (nm)'][i] = left.values[0]
    elif len(right) > 0 and len(left) == 0:
        pd_out['Process Size (nm)'][i] = right.values[0]
    elif len(right) == 0 and len(left) == 0:
        pd_out['Process Size (nm)'][i] = np.nan
    else:
        if left.values[0] == 'nan' and right.values[0] != 'nan':
            pd_out['Process Size (nm)'][i] = right.values[0]
        elif right.values[0] == 'nan' and left.values[0] != 'nan':
            pd_out['Process Size (nm)'][i] = left.values[0]
        elif left.values[0] in right.values:
            pd_out['Process Size (nm)'][i] = left.values[0]
        else:
            num += 1
            print('Conflict in Codename for', cur_name)
            print('Old:', left.values[0])
            print('New:', right.values)
            pd_out['Process Size (nm)'][i] = right.values[0]

print('Total conflicts in Process Size (nm):', num)
pd_out['Process Size (nm)'] = pd_out['Process Size (nm)'].astype(float)

Conflict in Codename for geforce rtx 4050 (laptop, 50w)
Old: 4.0
New: ['5.0']
Conflict in Codename for geforce rtx 4060 (laptop, 115w)
Old: 4.0
New: ['5.0']
Conflict in Codename for geforce rtx 4070 (laptop, 115w)
Old: 4.0
New: ['5.0']
Total conflicts in Process Size (nm): 3


In [11]:
# Architecture
pd_out['Architecture'] = np.nan
num = 0

for i in range(len(gpu_key)):
    cur_name = gpu_key['0'][i]
    left = old_gpu[old_gpu.index == cur_name]['Architecture'].astype(str)
    right_1 = new_gpu[new_gpu.index == cur_name]['Architecture'].astype(str)
    right_2 = new_gpu[new_gpu.index == cur_name]['GPU Name'].astype(str)

    if len(left) > 0 and len(right_1) == 0 and len(right_2) == 0:
        pd_out['Architecture'][i] = left.values[0]
    elif len(left) == 0:
        if len(right_1) > 0 and len(right_2) == 0:
            pd_out['Architecture'][i] = right_1.values[0]
        elif len(right_2) > 0 and len(right_1) == 0:
            pd_out['Architecture'][i] = right_2.values[0]
        elif len(right_1) == 0 and len(right_2) == 0:
            pd_out['Architecture'][i] = np.nan
        else:
            bin_not_nan = {}
            for value in right_2.values:
                if value != 'nan':
                    bin_not_nan[value] = bin_not_nan.get(value, 0) + 1
            if len(bin_not_nan) == 0:
                pd_out['Architecture'][i] = np.nan
            else:
                pd_out['Architecture'][i] = max(bin_not_nan, key=bin_not_nan.get)
            
    else:
        if left.values[0] == 'nan':
            bin_not_nan = {}
            for value in right_2.values:
                if value != 'nan':
                    bin_not_nan[value] = bin_not_nan.get(value, 0) + 1
            if len(bin_not_nan) == 0:
                pd_out['Architecture'][i] = np.nan
            else:
                pd_out['Architecture'][i] = max(bin_not_nan, key=bin_not_nan.get)
        elif left.values[0] != 'nan':
            if left.values[0] in right_1.values:
                pd_out['Architecture'][i] = left.values[0]
            elif left.values[0] in right_2.values:
                pd_out['Architecture'][i] = left.values[0]
            elif left.values[0] in right_1.values and left.values[0] in right_2.values:
                pd_out['Architecture'][i] = left.values[0]
            else:
                num += 1
                print('Conflict in Architecture for', cur_name)
                print('Old:', left.values[0])
                print('New:', right_1.values, right_2.values)
                pd_out['Architecture'][i] = right_2.values[0]

print('Total conflicts in Architecture:', num)

for i in range(len(pd_out)):
    if pd_out['Architecture'][i] == 'nan':
        pd_out['Architecture'][i] = np.nan

Conflict in Architecture for radeon pro 5600m (8gb hbm2)
Old: Navi
New: ['RDNA 1.0'] ['Navi 12']
Conflict in Architecture for radeon rx 640
Old: Polaris
New: ['GCN 4.0'] ['Polaris 23']
Conflict in Architecture for radeon rx 6700s
Old: Navi 22
New: ['RDNA 2.0'] ['Navi 23']
Conflict in Architecture for radeon rx 6800s
Old: Navi 22
New: ['RDNA 2.0'] ['Navi 23']
Conflict in Architecture for radeon rx vega 10
Old: Vega
New: ['GCN 5.0'] ['Raven-M']
Conflict in Architecture for arc a350m
Old: Alchemist
New: ['Generation 12.7'] ['DG2-128']
Conflict in Architecture for arc a370m
Old: Alchemist
New: ['Generation 12.7'] ['DG2-128']
Conflict in Architecture for arc a530m
Old: Alchemist
New: ['Generation 12.7'] ['DG2-256']
Conflict in Architecture for arc a550m
Old: Alchemist
New: ['Generation 12.7'] ['DG2-512']
Conflict in Architecture for arc a730m
Old: Alchemist
New: ['Generation 12.7'] ['DG2-512']
Conflict in Architecture for iris plus graphics g7
Old: Ice Lake
New: ['Generation 11.0'] ['Ice La

In [12]:
# Memory type
pd_out['Memory Type'] = np.nan
num = 0

for i in range(len(gpu_key)):
    cur_name = gpu_key['0'][i]
    left = old_gpu[old_gpu.index == cur_name]['Memory Type'].astype(str)
    right = new_gpu[new_gpu.index == cur_name]['Memory Type'].astype(str)
    if len(left) > 0 and len(right) == 0:
        pd_out['Memory Type'][i] = left.values[0]
    elif len(right) > 0 and len(left) == 0:
        pd_out['Memory Type'][i] = right.values[0]
    elif len(right) == 0 and len(left) == 0:
        pd_out['Memory Type'][i] = np.nan
    else:
        if left.values[0] == 'nan' and right.values[0] != 'nan':
            pd_out['Memory Type'][i] = right.values[0]
        elif right.values[0] == 'nan' and left.values[0] != 'nan':
            pd_out['Memory Type'][i] = left.values[0]
        elif left.values[0] in right.values:
            pd_out['Memory Type'][i] = left.values[0]
        else:
            num += 1
            print('Conflict in Memory Type for', cur_name)
            print('Old:', left.values[0])
            print('New:', right.values)
            pd_out['Memory Type'][i] = right.values[0]

print('Total conflicts in Memory Type:', num)

Total conflicts in Memory Type: 0


In [13]:
# Memory clock / Memory frequency
pd_out['Memory Frequency (MHz)'] = np.nan
num = 0

for i in range(len(gpu_key)):
    cur_name = gpu_key['0'][i]
    left = old_gpu[old_gpu.index == cur_name]['Memory Frequency'].astype(str)
    right = new_gpu[new_gpu.index == cur_name]['Memory Clock'].astype(str)
    if len(left) > 0 and len(right) == 0:
        pd_out['Memory Frequency (MHz)'][i] = left.values[0]
    elif len(right) > 0 and len(left) == 0:
        pd_out['Memory Frequency (MHz)'][i] = right.values[0]
    elif len(right) == 0 and len(left) == 0:
        pd_out['Memory Frequency (MHz)'][i] = np.nan
    else:
        if left.values[0] == 'nan' and right.values[0] != 'nan':
            pd_out['Memory Frequency (MHz)'][i] = right.values[0]
        elif right.values[0] == 'nan' and left.values[0] != 'nan':
            pd_out['Memory Frequency (MHz)'][i] = left.values[0]
        elif left.values[0] in right.values:
            pd_out['Memory Frequency (MHz)'][i] = left.values[0]
        else:
            num += 1
            print('Conflict in Memory Frequency for', cur_name)
            print('Old:', left.values[0])
            print('New:', right.values)
            pd_out['Memory Frequency (MHz)'][i] = right.values[0]

print('Total conflicts in Memory Frequency (MHz):', num)
pd_out['Memory Frequency (MHz)'] = pd_out['Memory Frequency (MHz)'].astype(float)

Conflict in Memory Frequency for radeon rx 5300m
Old: 14000.0
New: ['1750.0']
Conflict in Memory Frequency for radeon rx 5600m
Old: 12000.0
New: ['1500.0']
Conflict in Memory Frequency for radeon rx 640
Old: 6000.0
New: ['1500.0']
Conflict in Memory Frequency for radeon rx 6500m
Old: 18000.0
New: ['2250.0']
Conflict in Memory Frequency for radeon rx 6600m
Old: 2000.0
New: ['1750.0']
Conflict in Memory Frequency for radeon rx 6600s
Old: 16000.0
New: ['1750.0']
Conflict in Memory Frequency for radeon rx 6650m
Old: 18000.0
New: ['1750.0']
Conflict in Memory Frequency for radeon rx 6700s
Old: 16000.0
New: ['1750.0']
Conflict in Memory Frequency for radeon rx 6800s
Old: 16000.0
New: ['2000.0']
Conflict in Memory Frequency for radeon rx 6850m xt
Old: 18000.0
New: ['2250.0']
Conflict in Memory Frequency for radeon rx 7600m xt
Old: 18000.0
New: ['2250.0']
Conflict in Memory Frequency for radeon rx 7600s
Old: 16000.0
New: ['2000.0']
Conflict in Memory Frequency for radeon rx 7700s
Old: 18000.0


In [14]:
# Memory Size
pd_out['Memory Size'] = np.nan
num = 0

for i in range(len(gpu_key)):
    cur_name = gpu_key['0'][i]
    left = old_gpu[old_gpu.index == cur_name]['Memory Capacity'].astype(str)
    right = new_gpu[new_gpu.index == cur_name]['Memory Size'].astype(str)
    for j in range(len(right)):
        if right.values[j] != 'nan':
            right.values[j] = str(int(float(right.values[j]) / 1024))
    if len(left) > 0 and len(right) == 0:
        pd_out['Memory Size'][i] = left.values[0]
    elif len(right) > 0 and len(left) == 0:
        pd_out['Memory Size'][i] = right.values[0]
    elif len(right) == 0 and len(left) == 0:
        pd_out['Memory Size'][i] = np.nan
    else:
        if left.values[0] == 'nan' and right.values[0] != 'nan':
            pd_out['Memory Size'][i] = right.values[0]
        elif right.values[0] == 'nan' and left.values[0] != 'nan':
            pd_out['Memory Size'][i] = left.values[0]
        elif left.values[0] in right.values:
            pd_out['Memory Size'][i] = left.values[0]
        else:
            num += 1
            print('Conflict in Memory Size for', cur_name)
            print('Old:', left.values[0])
            print('New:', right.values)
            pd_out['Memory Size'][i] = right.values[0]

print('Total conflicts in Memory Size:', num)
pd_out['Memory Size'] = pd_out['Memory Size'].astype(float)

Conflict in Memory Size for radeon pro 5600m (8gb hbm2)
Old: 8.0
New: ['8']
Conflict in Memory Size for radeon rx 5300m
Old: 3.0
New: ['3']
Conflict in Memory Size for radeon rx 5600m
Old: 6.0
New: ['6']
Conflict in Memory Size for radeon rx 640
Old: 4.0
New: ['2']
Conflict in Memory Size for radeon rx 6500m
Old: 4.0
New: ['4']
Conflict in Memory Size for radeon rx 6600m
Old: 8.0
New: ['8']
Conflict in Memory Size for radeon rx 6600s
Old: 4.0
New: ['4']
Conflict in Memory Size for radeon rx 6650m
Old: 8.0
New: ['8']
Conflict in Memory Size for radeon rx 6700m
Old: 10.0
New: ['10']
Conflict in Memory Size for radeon rx 6700m
Old: 10.0
New: ['10']
Conflict in Memory Size for radeon rx 6700s
Old: 8.0
New: ['8']
Conflict in Memory Size for radeon rx 6800m
Old: 12.0
New: ['12']
Conflict in Memory Size for radeon rx 6800s
Old: 12.0
New: ['8']
Conflict in Memory Size for radeon rx 6850m xt
Old: 12.0
New: ['12']
Conflict in Memory Size for radeon rx 7600m xt
Old: 8.0
New: ['8']
Conflict in Mem

In [15]:
# Base Clock
pd_out['Base Clock'] = np.nan
num = 0

for i in range(len(gpu_key)):
    cur_name = gpu_key['0'][i]
    left = old_gpu[old_gpu.index == cur_name]['Base Frequency'].astype(str)
    right = new_gpu[new_gpu.index == cur_name]['Base Clock'].astype(str)
    if len(left) > 0 and len(right) == 0:
        pd_out['Base Clock'][i] = left.values[0]
    elif len(right) > 0 and len(left) == 0:
        pd_out['Base Clock'][i] = right.values[0]
    elif len(right) == 0 and len(left) == 0:
        pd_out['Base Clock'][i] = np.nan
    else:
        if left.values[0] == 'nan' and right.values[0] != 'nan':
            pd_out['Base Clock'][i] = right.values[0]
        elif right.values[0] == 'nan' and left.values[0] != 'nan':
            pd_out['Base Clock'][i] = left.values[0]
        elif left.values[0] in right.values:
            pd_out['Base Clock'][i] = left.values[0]
        else:
            num += 1
            print('Conflict in Base Clock for', cur_name)
            print('Old:', left.values[0])
            print('New:', right.values)
            pd_out['Base Clock'][i] = right.values[0]

print('Total conflicts in Base Clock:', num)
pd_out['Base Clock'] = pd_out['Base Clock'].astype(float)

Conflict in Base Clock for radeon 740m
Old: 1500.0
New: ['800.0']
Conflict in Base Clock for radeon 760m
Old: 1500.0
New: ['800.0' '800.0']
Conflict in Base Clock for radeon 780m
Old: 1500.0
New: ['800.0' '800.0']
Conflict in Base Clock for radeon pro 5600m (8gb hbm2)
Old: 1000.0
New: ['822.0']
Conflict in Base Clock for radeon rx 5300m
Old: 1181.0
New: ['1000.0']
Conflict in Base Clock for radeon rx 5600m
Old: 1190.0
New: ['1035.0']
Conflict in Base Clock for iris xe graphics g4
Old: 400.0
New: ['300.0']
Conflict in Base Clock for iris xe graphics g7 (80eu)
Old: 400.0
New: ['300.0']
Conflict in Base Clock for iris xe graphics g7 (80eu)
Old: 400.0
New: ['300.0']
Conflict in Base Clock for iris xe graphics g7 (96eu)
Old: 400.0
New: ['300.0']
Conflict in Base Clock for iris xe graphics g7 (96eu)
Old: 400.0
New: ['300.0']
Conflict in Base Clock for uhd graphics 600
Old: 300.0
New: ['200.0']
Conflict in Base Clock for uhd graphics 605
Old: 300.0
New: ['200.0']
Conflict in Base Clock for uh

In [16]:
# Memory Bus
pd_out['Memory Bus'] = np.nan
num = 0

for i in range(len(gpu_key)):
    cur_name = gpu_key['0'][i]
    left = old_gpu[old_gpu.index == cur_name]['Memory Bus'].astype(float).astype(str)
    right = new_gpu[new_gpu.index == cur_name]['Memory Bus'].astype(str)
    if len(left) > 0 and len(right) == 0:
        pd_out['Memory Bus'][i] = left.values[0]
    elif len(right) > 0 and len(left) == 0:
        pd_out['Memory Bus'][i] = right.values[0]
    elif len(right) == 0 and len(left) == 0:
        pd_out['Memory Bus'][i] = np.nan
    else:
        if left.values[0] == 'nan' and right.values[0] != 'nan':
            pd_out['Memory Bus'][i] = right.values[0]
        elif right.values[0] == 'nan' and left.values[0] != 'nan':
            pd_out['Memory Bus'][i] = left.values[0]
        elif left.values[0] in right.values:
            pd_out['Memory Bus'][i] = left.values[0]
        else:
            num += 1
            print('Conflict in Memory Bus for', cur_name)
            print('Old:', left.values[0])
            print('New:', right.values)
            pd_out['Memory Bus'][i] = right.values[0]

print('Total conflicts in Memory Bus:', num)
pd_out['Memory Bus'] = pd_out['Memory Bus'].astype(float)

Conflict in Memory Bus for radeon rx 6650m
Old: 256.0
New: ['128.0']
Conflict in Memory Bus for geforce gtx 1650 ti (laptop)
Old: 256.0
New: ['128.0' '128.0']
Total conflicts in Memory Bus: 2


In [17]:
# Release quarter
pd_out['Release quarter'] = np.nan
num = 0

for i in range(len(pd_out)):
    cur_name = pd_out['Name'][i]
    left = old_gpu[old_gpu.index == cur_name]['Released'].astype(str)
    right = new_gpu[new_gpu.index == cur_name]['Release Date'].astype(str)
    for j in range(len(right)):
        if right.values[j] != 'nan':
            right.values[j] = right.values[j].split('-')[0] + '-' + str((int(right.values[j].split('-')[1]) - 1) // 3 + 1)
            year = right.values[j].split('-')[0]
            quarter = right.values[j].split('-')[1]
            right.values[j] = str(float(year) + (float(quarter) - 1) / 4)
    if len(left) > 0 and len(right) == 0:
        pd_out['Release quarter'][i] = left.values[0]
    elif len(right) > 0 and len(left) == 0:
        pd_out['Release quarter'][i] = right.values[0]
    elif len(right) == 0 and len(left) == 0:
        pd_out['Release quarter'][i] = np.nan
    else:
        if left.values[0] == 'nan' and right.values[0] != 'nan':
            pd_out['Release quarter'][i] = right.values[0]
        elif right.values[0] == 'nan' and left.values[0] != 'nan':
            pd_out['Release quarter'][i] = left.values[0]
        elif left.values[0] in right.values:
            pd_out['Release quarter'][i] = left.values[0]
        else:
            num += 1
            print('Conflict in Release quarter for', cur_name)
            print('Old:', left.values[0])
            print('New:', right.values)
            pd_out['Release quarter'][i] = right.values[0]

print('Total conflicts in Release quarter:', num)
pd_out['Release quarter'] = pd_out['Release quarter'].astype(float)

Conflict in Release quarter for radeon 890m
Old: 2024.25
New: ['2024.5']
Conflict in Release quarter for radeon r3 (beema)
Old: 2014.25
New: ['2015.0' '2016.25']
Conflict in Release quarter for radeon rx 5300m
Old: 2019.5
New: ['2019.75']
Conflict in Release quarter for radeon rx 5600m
Old: 2020.0
New: ['2020.5']
Conflict in Release quarter for radeon rx 640
Old: 2019.5
New: ['2019.25']
Conflict in Release quarter for iris plus graphics g7
Old: 2019.25
New: ['2020.25']
Conflict in Release quarter for uhd graphics 630
Old: 2017.75
New: ['2018.25']
Conflict in Release quarter for uhd graphics 770
Old: 2022.25
New: ['2023.0' '2022.0']
Conflict in Release quarter for geforce mx130 (2gb gddr5)
Old: 2018.0
New: ['2017.75']
Conflict in Release quarter for geforce rtx 2050 (30w)
Old: 2022.0
New: ['2021.75']
Total conflicts in Release quarter: 10


In [18]:
# TDP
pd_out['Power Consumption'] = np.nan
num = 0

for i in range(len(gpu_key)):
    cur_name = gpu_key['0'][i]
    left = old_gpu[old_gpu.index == cur_name]['Power Consumption'].astype(str)
    right = new_gpu[new_gpu.index == cur_name]['TDP'].astype(str)
    if len(left) > 0 and len(right) == 0:
        pd_out['Power Consumption'][i] = left.values[0]
    elif len(right) > 0 and len(left) == 0:
        pd_out['Power Consumption'][i] = right.values[0]
    elif len(right) == 0 and len(left) == 0:
        pd_out['Power Consumption'][i] = np.nan
    else:
        if left.values[0] == 'nan' and right.values[0] != 'nan':
            pd_out['Power Consumption'][i] = right.values[0]
        elif right.values[0] == 'nan' and left.values[0] != 'nan':
            pd_out['Power Consumption'][i] = left.values[0]
        elif left.values[0] in right.values:
            pd_out['Power Consumption'][i] = left.values[0]
        else:
            num += 1
            print('Conflict in Power Consumption for', cur_name)
            print('Old:', left.values[0])
            print('New:', right.values)
            pd_out['Power Consumption'][i] = right.values[0]

print('Total conflicts in Power Consumption:', num)
pd_out['Power Consumption'] = pd_out['Power Consumption'].astype(float)

Conflict in Power Consumption for radeon 660m
Old: 45.0
New: ['40.0' '40.0']
Conflict in Power Consumption for radeon 680m
Old: 45.0
New: ['50.0' '50.0']
Conflict in Power Consumption for radeon 740m
Old: 30.0
New: ['15.0']
Conflict in Power Consumption for radeon 760m
Old: 54.0
New: ['15.0' '15.0']
Conflict in Power Consumption for radeon 780m
Old: 54.0
New: ['15.0' '15.0']
Conflict in Power Consumption for radeon rx 5300m
Old: 65.0
New: ['85.0']
Conflict in Power Consumption for arc a350m
Old: 35.0
New: ['25.0']
Conflict in Power Consumption for arc a370m
Old: 50.0
New: ['35.0']
Conflict in Power Consumption for arc a530m
Old: 95.0
New: ['65.0']
Conflict in Power Consumption for arc a550m
Old: 80.0
New: ['60.0']
Conflict in Power Consumption for arc a730m
Old: 120.0
New: ['80.0']
Conflict in Power Consumption for iris plus graphics g7
Old: 28.0
New: ['15.0']
Conflict in Power Consumption for iris xe graphics g7 (80eu)
Old: 28.0
New: ['15.0']
Conflict in Power Consumption for iris xe 

In [19]:
cols = ['Process Size (nm)', 'Architecture', 'Memory Type', 'Memory Frequency (MHz)', 'Memory Size', 'Base Clock', 'Memory Bus', 'Release quarter', 'Power Consumption']
pd_out = pd_out.dropna(subset=cols, how='all')

In [20]:
# Almost all nan are shared mem, which mean dedicated gpu ram doesn't exist, set to 0
pd_out['Memory Frequency (MHz)'].fillna(0, inplace=True)

# Almost all nan are shared mem, which mean dedicated gpu ram doesn't exist, set to 0
pd_out['Memory Size'].fillna(0, inplace=True)

# Almost all nan are shared mem, which mean mem bus does not exist, set to 0
pd_out['Memory Bus'] = pd_out['Memory Bus'].fillna(0)

In [21]:
# Rename all columns to have GPU: prefix
for column in pd_out.columns:
    pd_out.rename(columns={column: 'GPU: ' + column}, inplace=True)

In [22]:
pd_out.to_csv(data_path + 'gpu_merged.csv', index=False)